Randomly generate the initial hyperparameters for PINN-based field model training.

Only execute in the first loop of the optimiztion process.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import json


for i in range(12):
    regularization_lambda_seed = np.random.random()
    regularization_lambda = 0.02 * pow(50, regularization_lambda_seed * 2 - 1) # 0.0004~1
    regularization_lambda = 0.002

    diffusion_k_seed = np.random.random()
    diffusion_k = 0.01 * pow(10, diffusion_k_seed * 2 - 1) # 0.001~0.1
    
    sprayer_model_variance_num_seed = np.random.random()
    sprayer_model_variance_num = 0.15 * pow(3, sprayer_model_variance_num_seed * 2 - 1) # 0.05~0.45
    
    sprayer_lambda_seed = np.random.random()
    sprayer_lambda = 0.00002 * pow(50, sprayer_lambda_seed * 2 - 1) # 0.0000004~0.001
    sprayer_lambda = 0

    # 创建字典
    info_dict = {'diffusion_k': diffusion_k,\
                 'sprayer_model_variance_num': sprayer_model_variance_num,\
                 'sprayer_lambda': sprayer_lambda,\
                 'regularization_lambda': regularization_lambda}
    info_json = json.dumps(info_dict,sort_keys = False, indent = len(info_dict), separators=(',', ': '))
    f = open(str(i + 1) + '\info.json', 'w')
    f.write(info_json)
    f.close()

Read result data here.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import json
i = 3
time_slice = 1
x_slice = 0.1
y_slice = 0.1
lat_range = 30 # latitude grid range (Unit: km)
lon_range = 30 # longitude grid range (Unit: km)
lb_x = 0.25
ub_x = 14.25
lb_y = 0.25
ub_y = 9.25
lb_x = int(lb_x/x_slice - x_slice + 0.0001)
lb_y = int(lb_y/y_slice - y_slice + 0.0001)
ub_x = int(ub_x/x_slice - x_slice + 0.0001)
ub_y = int(ub_y/y_slice - y_slice + 0.0001)
time_start = pd.Timestamp('2022-04-01 00:00:00+08:00')
time_start_2 = pd.Timestamp('2022-09-23 00:00:00+08:00')
start_time_2 = int((time_start_2 - time_start).total_seconds()/60)
refer_n = 12
train_range = 360
test_range = 180
update_time_range = 60

name_1 = []
name_1.append("1")
name_1.append("2")
name_1.append("3")
name_1.append("4")
name_1.append("5")
name_1.append("6")
name_1.append("7")
name_1.append("8")
name_1.append("9")
name_1.append("10")
name_1.append("11")
name_1.append("12")


name_2 = "result"

name_3 = "data_test_"

name_4 = "data_static_"

total_t_cal = []
info_data = []

print("exclude the rainy days...")
rain_time = ['2022-09-25','2022-09-27','2022-09-28',
            '2022-10-04','2022-10-05','2022-10-06','2022-10-07','2022-10-08','2022-10-26',
            '2022-11-12','2022-11-14','2022-11-16','2022-11-17','2022-11-18','2022-11-21','2022-11-22','2022-11-28','2022-11-29']
rain_num_range = []
i_pre = start_time_2 - 1440
for i in rain_time:
    rain_t = int((pd.Timestamp(i + ' 00:00:00+0800') - time_start).total_seconds() / 60)
    rain_num_range.append(range(i_pre + 1440 + train_range, rain_t - test_range, update_time_range))
    i_pre = rain_t
    
break_e = 0
data_result_m = [{} for i in range(12)]
for i in range(12):
    print("Experiment " + name_1[i] + ":")
    break_e = 0
    for root, dirs, files in os.walk(name_1[i] + '/' + name_2):
        for file in files:
            file_name = os.path.join(root, file).split("_")[2]
            if file_name[-4:] == ".csv":
                t = int(file_name[:-4])
                data_result_m[i][t] = pd.read_csv(os.path.join(root, file))
    print("The data in experiment %d have been end at time %d"%\
      (i, max(data_result_m[i].keys())))

            
    f = open(str(i + 1) + '\info.json', 'r')
    info_data.append(json.load(f))
    print("###############################")

    

Experiment 1:
The data in experiment 0 have been end at time 106140
###############################
Experiment 2:
The data in experiment 1 have been end at time 106140
###############################
Experiment 3:
The data in experiment 2 have been end at time 106140
###############################
Experiment 4:
The data in experiment 3 have been end at time 29940
###############################
Experiment 5:
The data in experiment 4 have been end at time 106140
###############################
Experiment 6:
The data in experiment 5 have been end at time 106140
###############################
Experiment 7:
The data in experiment 6 have been end at time 106140
###############################
Experiment 8:
The data in experiment 7 have been end at time 106140
###############################
Experiment 9:
The data in experiment 8 have been end at time 106140
###############################
Experiment 10:
The data in experiment 9 have been end at time 106140
###############################


Compute metric of result data here

In [2]:
end_t = 106140
test_range = 60
    
for k in range(12):
    print("Experiment " + name_1[k] + ":")
    print(info_data[k])
    data_m = data_result_m[k]
    count_sum = 0
    count_sum_abs = 0
    count_denom = 0
    count_number = 0
    for key, value in data_m.items():
        if key > end_t:
            continue
        data_test = value.loc[(value.time < key + test_range),:]
        count_number += data_test.shape[0]
        count_sum += ((data_test.pm2d5_after_cal - data_test.prediction_result)**2).sum()
        count_sum_abs += (abs(data_test.pm2d5_after_cal - data_test.prediction_result)).sum()
        count_denom += ((data_test.pm2d5_after_cal)).sum()
#             print(i,j,count_number)
    print("test_num:" + str(count_number))
    print("RMSE:" + str(pow(count_sum/count_number,0.5)))
    print("MSE:" + str(pow(count_sum_abs/count_number,1)))
    print("MAPE:" + str(pow(count_sum_abs/count_denom,1)))
    info_data[k]['RMSE'] = pow(count_sum/count_number,0.5)
    info_data[k]['test_num'] = count_number
    print("###############################")



Experiment 1:
{'diffusion_k': 0.0998749185957073, 'sprayer_model_variance_num': 0.3988680261268537, 'sprayer_lambda': 1.353466086307772e-06, 'regularization_lambda': 0.002, 'RMSE_predict': 20.149140041904193, 'RMSE_std': 0.9757379594756804, 'acq': 3.035012292930497e-29}
test_num:389965
RMSE:9.765764105893984
MSE:2.0038130659415407
MAPE:0.07125317906569657
###############################
Experiment 2:
{'diffusion_k': 0.005531938730578852, 'sprayer_model_variance_num': 0.39492494143914025, 'sprayer_lambda': 7.094616696148556e-06, 'regularization_lambda': 0.002, 'RMSE_predict': 18.49647674339275, 'RMSE_std': 0.8367630267728245, 'acq': 1.4751984478608124e-28}
test_num:389965
RMSE:10.898768396799062
MSE:2.006599121012279
MAPE:0.07135224782825547
###############################
Experiment 3:
{'diffusion_k': 0.036228751866988186, 'sprayer_model_variance_num': 0.1922799789703314, 'sprayer_lambda': 0.0002607247349531893, 'regularization_lambda': 0.002, 'RMSE_predict': 13.599256640592705, 'RMSE_

Save result metric here

In [3]:
test_num_min = 389965
info_data = pd.DataFrame(info_data)
info_data = info_data.loc[info_data.test_num >= test_num_min,:]
info_json = json.dumps(list(info_data.T.to_dict().values()),sort_keys = False, indent = info_data.shape[0], separators = (',', ': '))
f = open('info_result_4.json', 'w')
f.write(info_json)
f.close()

Read the up-to-now result metric here

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import json
info_data = []
for i in range(6):
    f = open('info_result_' + str(i+1) + '.json', 'r')
    info_data.extend(json.load(f))

Construct the summit file based on the document template. 

Note: This process may vary depending on the machine being used. 

Please customize the code accordingly to fit your specific requirements.


In [1]:
import os
import shutil
server_number = 3
task_on_server_number = 4
initial_name = "/submit.sh"
task_name = ["/submit.sh", "/1.sh", "/GPRT.sh"]
for i in range(server_number):
    tree_name = str(i * task_on_server_number + 1)
    shutil.copytree('-1', tree_name)
    os.rename(tree_name + initial_name, tree_name + task_name[i])
    for j in range(1, task_on_server_number):
        shutil.copytree(tree_name, str(i * task_on_server_number + j + 1))
        


Do the Bayesian Optimization and generate the next training hyperparameters

In [4]:
import gpflow
import scipy.stats as st
GPR_sample_num = 50
Acq_lambda = 1

b = pd.DataFrame(info_data)
X_list = [0,1,2]
Y_list = 4
lengthscale = [1 for i in X_list]
X = pd.DataFrame(b.iloc[:,X_list])
Y = pd.DataFrame(b.iloc[:,Y_list])
X_n = (X - X.mean())/X.std()
Y_mean = float(Y.mean())
Y_std = float(Y.std())
Y_n = (Y - Y_mean)/Y_std
k = gpflow.kernels.RBF(variance = 1, lengthscales = lengthscale)
model_GP = gpflow.models.GPR(data=(X_n, Y_n), kernel=k)  
opt = gpflow.optimizers.Scipy()
i_maxiter = 1000
while True:
    try:
        opt_logs = opt.minimize(model_GP.training_loss, model_GP.trainable_variables, options=dict(maxiter=i_maxiter))
        break
    except KeyboardInterrupt:
        raise
    except:
        print("%f maxiter is too large"%(i_maxiter))
        X = pd.DataFrame(b.iloc[:,X_list])
        Y = pd.DataFrame(b.iloc[:,Y_list])
        X_n = (X - X.mean())/X.std()
        Y_mean = float(Y.mean())
        Y_std = float(Y.std())
        Y_n = (Y - Y_mean)/Y_std
        k = gpflow.kernels.RBF(variance = 1, lengthscales = lengthscale)
        model_GP = gpflow.models.GPR(data=(X_n, Y_n), kernel=k)  
        opt = gpflow.optimizers.Scipy()
        i_maxiter = int(i_maxiter / 1.5)
        continue
# data_result = itertools.product(range( max(t - update_span,0), t + 1),\
#                                range(lb_x, ub_x + 1),\
#                                 range(lb_y, ub_y + 1),\
#                                 [0],\
#                                 [0])
# data_result = pd.DataFrame([x for x in data_result])
# data_result.columns = ['time', 'x', 'y','pm2d5','i_error']
# D_result, var = model_GP.predict_f(np.asarray(data_result.iloc[:,[0,1,2]]).astype('float64'))

info_total = []

for i in range(GPR_sample_num):
    regularization_lambda_seed = np.random.random()
    regularization_lambda = 0.02 * pow(50, regularization_lambda_seed * 2 - 1) # 0.0004~1
    regularization_lambda = 0.002

    diffusion_k_seed = np.random.random()
    diffusion_k = 0.01 * pow(10, diffusion_k_seed * 2 - 1) # 0.001~0.1
    
    sprayer_model_variance_num_seed = np.random.random()
    sprayer_model_variance_num = 0.15 * pow(3, sprayer_model_variance_num_seed * 2 - 1) # 0.05~0.45
    
    sprayer_lambda_seed = np.random.random()
    sprayer_lambda = 0.00002 * pow(50, sprayer_lambda_seed * 2 - 1) # 0.0000004~0.001
    # sprayer_lambda = 0

    # 创建字典
    info_dict = {'diffusion_k': diffusion_k,\
                 'sprayer_model_variance_num': sprayer_model_variance_num,\
                 'sprayer_lambda': sprayer_lambda,\
                 'regularization_lambda': regularization_lambda}

#     info_json = json.dumps(info_dict,sort_keys = False, indent = len(info_dict), separators=(',', ': '))
    info_total.append(info_dict)
#     info_json = json.dumps(info_dict,sort_keys=False, indent=7, separators=(',', ': '))
#     f = open(str(i + 1) + '\info.json', 'w')
#     f.write(info_json)
#     f.close()

info_total = pd.DataFrame(info_total)
Z = info_total.iloc[:,X_list]
Z_n = (Z - X.mean())/X.std()
D_result, var = model_GP.predict_f(np.asarray(Z_n).astype('float64'))
D_result = D_result * Y_std + Y_mean
info_total['RMSE_predict'] = np.array(D_result)
info_total['RMSE_std'] = pow(np.array(var), 0.5)

print("info_total:")
print(info_total)

f_min = Y.min()[0]

improvement =  f_min - info_total['RMSE_predict']
print("improvement:")
print(improvement)
# info_total['acq'] = info_total['after_cal_average'] - Acq_lambda * info_total['after_cal_average_var']
info_total['acq'] = improvement * st.norm.cdf(improvement / info_total['RMSE_std'])\
                    + info_total['RMSE_std'] * st.norm.pdf(improvement / info_total['RMSE_std'])
print("info_total['acq']:")
print(info_total['acq'])

info_dict_total = info_total.sort_values('acq').iloc[-12:,:].reset_index(drop = True).T.to_dict()

for i in range(12):
    info_dict = info_dict_total[i]
    info_json = json.dumps(info_dict,sort_keys=False, indent=len(info_dict), separators=(',', ': '))
    f = open(str(i + 1) + '\info.json', 'w')
    f.write(info_json)
    f.close()
# sort
# sub
# for i in range(12):
#     info_json = json.dumps(info_dict,sort_keys=False, indent=7, separators=(',', ': '))
#     f = open(str(i + 1) + '\info.json', 'w')
#     f.write(info_json)
#     f.close()

info_total:
    diffusion_k  sprayer_model_variance_num  sprayer_lambda  \
0      0.002102                    0.205756    8.991854e-05   
1      0.018925                    0.196554    1.224150e-04   
2      0.001664                    0.050676    5.714241e-07   
3      0.001214                    0.202239    1.147071e-04   
4      0.041286                    0.215622    7.647207e-06   
5      0.079051                    0.050364    2.247166e-06   
6      0.003442                    0.323644    2.060158e-05   
7      0.053922                    0.125151    4.838475e-05   
8      0.076808                    0.065912    3.361847e-06   
9      0.056763                    0.338641    2.162871e-05   
10     0.061938                    0.313771    8.143064e-04   
11     0.084386                    0.231499    3.176773e-06   
12     0.052729                    0.206547    1.161581e-04   
13     0.002444                    0.169064    1.747489e-06   
14     0.032427                    0.118353